In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import socket
import json
from collections import deque
import threading
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt

# Create a Dash web application
app = dash.Dash(__name__)

# Define the server and port to listen to
HOST = 'localhost'  # Replace with your server address
PORT = 12345  # Replace with your server port

# Create placeholders for data using deque to store a limited number of data points
data_limit = 1000  # Adjust the limit as needed
loss_data = [deque(maxlen=data_limit) for _ in range(4)]
image_data = [deque(maxlen=data_limit) for _ in range(4)]

# Define the layout of the app
app.layout = html.Div([
    html.H1("Real-Time Data Visualization"),

    dcc.Graph(id='loss-plot'),

    dcc.Graph(id='image-plot-1'),
    dcc.Graph(id='image-plot-2'),
    dcc.Graph(id='image-plot-3'),
    dcc.Graph(id='image-plot-4'),

    dcc.Interval(
        id='interval-component',
        interval=1000,  # Update every 1 second
        n_intervals=0
    ),
])

def socket_server_thread():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        s.listen()

        print(f"Listening on {HOST}:{PORT}...")
        conn, addr = s.accept()
        with conn:
            print(f"Connected by {addr}")
            data_buffer = b""  # Initialize an empty byte buffer

            while True:
                data_chunk = conn.recv(1024)
                if not data_chunk:
                    break

                # Append the received chunk to the buffer
                data_buffer += data_chunk

                # Check if the buffer contains a complete JSON message
                try:
                    decoded_data = json.loads(data_buffer.decode('utf-8'))
                    data_buffer = b""  # Reset the buffer
                    # Process the received data (e.g., update plots)
                    for i in range(4):
                        loss_data[i].append({'x': decoded_data['timestamp'], 'y': decoded_data[f'loss_{i+1}']})
                        image_data[i] = (decoded_data[f'image_{i+1}'])
                except json.JSONDecodeError:
                    # Incomplete JSON message, continue to receive data
                    continue


# Start the socket server thread
socket_thread = threading.Thread(target=socket_server_thread)
socket_thread.daemon = True  # Allow the thread to exit when the main program exits
socket_thread.start()

# Define callback to update loss plot
@app.callback(
    Output('loss-plot', 'figure'),
    Input('interval-component', 'n_intervals')
)
def update_loss_plot(n):
    # Define the desired line colors
    cell_colors = [(0, 255, 0), (255, 0, 255), (0, 0, 255), (255, 255, 0)]

    # Create a Plotly figure for loss plot with 4 lines
    fig = go.Figure()
    for i in range(4):
        fig.add_trace(go.Scatter(
            x=[point['x'] for point in loss_data[i]],
            y=[point['y'] for point in loss_data[i]],
            mode='lines',
            name=f'Loss {i+1}',
            line=dict(color=f'rgb{cell_colors[i]}')  # Set the line color using the predefined colors
        ))

    fig.update_layout(title='Loss Plot')
    return fig

# Define callbacks to update image plots
def create_image_plot_callback(cell_number):
    @app.callback(
        Output(f'image-plot-{cell_number}', 'figure'),
        Input('interval-component', 'n_intervals')
    )
    def update_image_plot(n):
        try:
            # Convert the deque to a list
            image_list = list(image_data[cell_number - 1])  # Adjusted to use cell_number
            image_array = np.array(image_list).reshape(41, 41)
            print("array_start")
            print(image_array)
            # Rotate the image array by 90 degrees to the right
            image_array = np.rot90(image_array, k=-1)
                        # Apply a horizontal flip (left to right)
            image_array = np.flip(image_array, axis=1)

            # Apply a vertical flip (top to bottom)
            image_array = np.flip(image_array, axis=0)


            heatmap_size = image_array.shape[0]
            print("array_end , Array Shape: " + str(image_array.shape))
            # Define the figure with a heatmap
            figure = {
                'data': [
                    go.Heatmap(
                        z=image_array,
                        colorscale='Viridis',  # You can choose a different colorscale
                    )
                ],
                'layout': go.Layout(
                    xaxis=dict(title='X Axis', range=[0, heatmap_size]),
                    yaxis=dict(title='Y Axis', range=[0, heatmap_size]),
                    title=f'Cell Vision Field [Cell Number : {cell_number}]'
                )
            }
            figure['layout']['yaxis']['scaleanchor'] = 'x'

            return figure

        except Exception as e:
            # Handle any exceptions gracefully (e.g., empty or invalid data)
            print(f"Error in update_image_plot for cell {cell_number}: {str(e)}")
            return []

# Create a callback for each cell
for i in range(4):
    create_image_plot_callback(i + 1)


if __name__ == '__main__':
    app.run_server(debug=False, port=45123)


Listening on localhost:12345...


<IPython.core.display.Javascript object>

In [ ]:
!pip install dash


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7
